In [1]:
import requests
import html
import random


In [2]:
# Vamos a utilizar la API de OPEN TRIVIA DATABASE (https://opentdb.com/) para generar preguntas 
# aleatorias sobre categorías específicas.
 
# Obtenemos un nuevo token de sesión para que las preguntas no se repitan:

def get_session_token():
    response = requests.get("https://opentdb.com/api_token.php?command=request")
    if response.status_code == 200:
        data = response.json()
        if data['response_code'] == 0:
            return data['token']
    return None

# Cuando el token se agota y no puede dar más preguntas, se resetea:

def reset_session_token(token):
    url = f"https://opentdb.com/api_token.php?command=reset&token={token}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data['response_code'] == 0  # Retorna True si el reset fue exitoso
    return False

In [3]:
# Consultamos todas las categorías disponibles de la API.

url = "https://opentdb.com/api_category.php"
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    categories = data['trivia_categories']



In [6]:
# Agrupamos las categorías de forma genérica en un diccionario:

generic_categories = {
    'Science': [],
    'Audiovisual': [],
    'History': [],
    'Arts and Books': [],
    'Sports': [],
    'General_Culture': [],
    'Animals':[],
}

# Mapeamos las categorías
category_map = {
    17: 'Science',#'Science & Nature'
    18: 'Science',#'Science: Computers'
    19: 'Science',#'Science: Mathematics'
    30: 'Science',#'Science: Gadgets'
    11: 'Audiovisual',#'Entertainment: Film'
    12: 'Audiovisual',#'Entertainment: Music'
    14: 'Audiovisual',#'Entertainment: Television'
    31: 'Audiovisual',#'Entertainment: Japanese Anime & Manga'
    32: 'Audiovisual',#'Entertainment: Cartoon & Animations'
    13: 'Audiovisual',#'Entertainment: Musicals & Theatres'
    20: 'History', #'Mythology'
    23: 'History',#'History'
    24: 'History',#'Politics'
    28: 'History',#'Vehicles'
    22: 'History',#'Geography'
    27: 'Animals',#'Animals'
    10: 'Arts and Books',#'Entertainment: Books'
    25: 'Arts and Books',#'Art'
    29: 'Arts and Books',#'Entertainment: Comics'
    21: 'Sports',#'Sports'
    9: 'General_Culture', # 'General Knowledge'
    26: 'General_Culture',#'Celebrities'
    15: 'General_Culture',#'Entertainment: Video Games'
    16: 'General_Culture' #'Entertainment: Board Games'
}

for category in categories:
    id_ = category['id']
    if id_ in category_map:
        generic_categories[category_map[id_]].append(category)


In [7]:
# Para obtener preguntas sólo de la categoría genérica especificada:

def get_random_category_id(generic_categories, cat):
    # Verifica si la categoría especificada existe en el diccionario
    if cat in generic_categories:
        # Obtiene la lista de categorías para la categoría especificada
        categories = generic_categories[cat]
        
        # Elige un id aleatorio de entre las categorías especificadas
        if categories:  # Verifica que la lista no esté vacía
            random_category = random.choice(categories)
            return random_category['id']
    return None


In [8]:
def get_trivia_question(token, category):
    # Establece el amount=1 para obtener solo una pregunta
    url = f"https://opentdb.com/api.php?amount=1&token={token}&category={category}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if data['response_code'] == 0:
            question_data = data['results'][0]
            
            # Extrae la pregunta y la decodifica
            question = html.unescape(question_data['question'])
            
            # Extrae las opciones de respuesta
            options = question_data['incorrect_answers']
            options.append(question_data['correct_answer'])
            options = [html.unescape(option) for option in options]
            
            # Desordena las opciones aleatoriamente para no mostrar siempre la correcta en la misma posición
            random.shuffle(options)
            return question, options, html.unescape(question_data['correct_answer'])
        
            #Si se han agotado las preguntas y hay que resetear el token:
        elif data['response_code'] == 4:
            print("No more questions available. Reseting token")
            reset_successful = reset_session_token(token)
            if reset_successful:
                print("Token successfully reset.")
            else:
                print("Error obtaining a token for this session.")

    return 
    

        

In [9]:
# Ahora, asignaremos las categorías a objetos cotidianos, para poder implementarlas en el juego:

#"Science": [ 'banana',   ], # 'microwave', 'computer mouse', 'laptop', 'keyboard', 'cell phone', , 'oven', , 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',

#"Audiovisual": [  'headphones' ], #'remote', 'camera','tv', 

#"History": [ 'airplane',  ], #'train','car' 'motorcycle', 'bus', 'boat', 'ship', 'bicycle', 'truck', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter',

#"Arts and Books": [ 'book' ],#'vase', 'teddy bear', 'pencil', 'brush' , 'scissors'

#"Sports": [  'sports ball'  ], #  ,  'kite','baseball bat','baseball glove', 'skateboard', 'surfboard','frisbee', 'tennis racket', 'skis', 'snowboard',
#"Animals": ['cat', 'dog'] #'bird',  'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe',
#"General Culture": [ 'cup', 'bottle',  ]  #person',  'bench',  'backpack', 
# 'umbrella', 'handbag', 'tie', 'suitcase', , 'wine glass',  'fork', 'knife', 
# 'spoon', 'bowl',  'chair', 'couch', 'potted plant', 'bed', 'dining table', 
# 'toilet', 'sink', 'refrigerator', 'toaster', 'clock', 'hair drier', 'toothbrush'

objeto_categorias = {
    'Science': ['banana'],
    'Audiovisual': ['headphones'],
    'History': ['airplane'],
    'Arts and Books': ['book'],
    'Animals':['cat', 'dog'],
    'Sports': ['basketball'],
    'General_Culture': ['cup', 'bottle']}

def objeto_a_cat(objeto):
    for category, objetos in objeto_categorias.items():
        if objeto in objetos:
            return category





In [10]:
def new_question(objeto):
    token = get_session_token()  # Aseguramos tener el token
    categoria = objeto_a_cat(objeto)
    category_id = get_random_category_id(generic_categories, categoria)

    if token:
        question, options, correct_answer = get_trivia_question(token,category_id)
        if question:
            print(f'Oh! You found a {objeto}! Get ready for a question about {categoria}!:')
            print(f"Question: {question}")
            print("Options:")
            for i, option in enumerate(options, start=1):
                print(f"{i}. {option}")
            
            print(f"Correct answer: {correct_answer}")  # Solo para pruebas
            answer = input("What's the number for the correct answer?")

            print(options[int(answer)-1])
            if options[int(answer)-1] == correct_answer:
                print("That's correct! Well done :)")
            else:
                print("I don't think so...")
                print(f"My records say otherwise, the correct answer is {correct_answer}.")
        else:
            print("No more questions available.")
    else:
        print("Error obtaining a token for this session.")
    return

In [ ]:
import cv2
import os

def verificar_imagenes(directorio, max_pixels=89478485):
    imagenes_grandes = []

    imagengrande = 1
    # Recorrer directorios y archivos
    for root, dirs, files in os.walk(directorio):
        for file in files:
            file_path = os.path.join(root, file)
            
            try:
                # Cargar la imagen
                print(imagengrande)
                image = cv2.imread(file_path)
                imagengrande +=1
                if image is not None:
                    height, width, channels = image.shape
                    num_pixels = height * width

                    if num_pixels > max_pixels:
                        imagenes_grandes.append((file_path, num_pixels))
                        

            except Exception as e:
                print(f"Error procesando archivo {file_path}: {e}")

    return imagenes_grandes

# Directorio de inicio
directorio_base = './imagenes'

# Verificar imágenes y obtener resultados
imagenes_con_riesgo = verificar_imagenes(directorio_base)

# Mostrar los resultados
if imagenes_con_riesgo:
    print("Imágenes con riesgo de desbordamiento debido a su tamaño:")
    for img_path, num_pixels in imagenes_con_riesgo:
        print(f"Archivo: {img_path}, Número de píxeles: {num_pixels}")
else:
    print("No se han encontrado imágenes con riesgo de desbordamiento.")

In [ ]:
def listar_imagenes_y_etiquetas(base_path, subfolder):
    image_paths = []
    labels = []
    class_names = os.listdir(os.path.join(base_path, subfolder))  # Lista las clases

    for label, class_name in enumerate(class_names):
        class_path = os.path.join(base_path, subfolder, class_name)
        image_files = os.listdir(class_path)

        for image_file in image_files:
            image_paths.append(os.path.join(class_path, image_file))
            labels.append(label)
    
    return image_paths, labels, class_names

base_dir = './imagenes'
train_image_paths, train_labels, class_names = listar_imagenes_y_etiquetas(base_dir, 'train')
val_image_paths, val_labels, _ = listar_imagenes_y_etiquetas(base_dir, 'validation')

print(f"Clases: {class_names}")
print(f"Rutas de entrenamiento: {train_image_paths[:5]}")
print(f"Etiquetas de entrenamiento: {train_labels[:5]}")

Clases: ['airplane', 'banana', 'basketball', 'book', 'bottle', 'cat', 'cup', 'dog', 'headphones']
Rutas de entrenamiento: ['./imagenes\\train\\airplane\\airplane_0.jpg', './imagenes\\train\\airplane\\airplane_000.jpg', './imagenes\\train\\airplane\\airplane_001.jpg', './imagenes\\train\\airplane\\airplane_0010.jpg', './imagenes\\train\\airplane\\airplane_0011.jpg']
Etiquetas de entrenamiento: [0, 0, 0, 0, 0]


In [ ]:
# Ejemplo
objeto = 'book'  # Este es el objeto que detectaremos --> 
#Cambiar para que objeto desbloquee tipología de preguntas
new_question(objeto)

Oh! You found a book! Get ready for a question about Arts and Books!:
Question: How many paint and pastel versions of "The Scream" is Norwegian painter Edvard Munch believed to have produced?
Options:
1. 3
2. 1
3. 2
4. 4
Correct answer: 4
4
That's correct! Well done :)


In [ ]:
# objeto --> desbloquea categoría B
# Categorias disponibles A y B
# New question aleatoria entre subcategorias de A y de B
# y sigues con tu rollo --> Investigar habitacion para conseguir puntos respondiendo a X preguntas
# La siguiente puerta no se desbloquea aunque enseñes objeto si no tienes X puntos
# enigma --> objeto como respuesta --> desbloquea categoria C....

In [ ]:
objetitos = ['banana','basketball','book','cup','dog','headphones','airplane','cat','bottle']


Oh! You found a banana! Get ready for a question about Science!:
Question: When did the CD begin to appear on the consumer market?
Options:
1. 1982
2. 1972
3. 1962
4. 1992
Correct answer: 1982
1982
That's correct! Well done :)
siiiguientee
Oh! You found a headphones! Get ready for a question about Audiovisual!:
Question: In Alfred Hitchcock's film 'Psycho' it is said he used chocolate syrup to simulate the blood in the famous shower scene from 
Options:
1. True
2. False
Correct answer: True


ValueError: invalid literal for int() with base 10: ''